In [1]:
from copy import deepcopy, copy
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit.circuit.library import MCMT

from context import tools as t

plt.style.use('science')

In [2]:
%%javascript
IPython.notebook.kernel.execute('NB_NAME = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
NB_NAME

NameError: name 'NB_NAME' is not defined

In [ ]:
def get_filename_png():
    now = datetime.now()
    return now.strftime('figures/%Y-%m-%d--%H:%M:%S-')+NB_NAME[:-6]+'.png'

def get_filename_txt():
    now = datetime.now()
    return now.strftime('data/%Y-%m-%d--%H:%M:%S-')+NB_NAME[:-6]+'.txt'

def circ(n_qubits):
    qr = qk.QuantumRegister(n_qubits, 'q')
    qc = qk.QuantumCircuit(qr)
    return qr, qc

def sort(dic):
    return sorted(dic.items())

def isdagger(gate, dag_gate):
    qr = qk.QuantumRegister(gate.num_qubits, 'q')
    qc = qk.QuantumCircuit(qr)
    qc.compose(gate, qr, inplace=True)
    qc.compose(dag_gate, qr, inplace=True)
    qc.measure_all()
    counts = t.counts(qc)
    return list(counts.keys()) == ['0'*gate.num_qubits]

def all_combinations(s):
    arr = np.array(list(s), dtype=object)
    dots_pos = np.where(arr == '.')[0]
    bit_lst = [list(''.join(str(bit) for bit in bits)) for bits in product([0, 1], repeat=len(dots_pos))]
    result = []
    for bit in bit_lst:
        arr[dots_pos] = bit
        result.append(''.join(str(b) for b in arr))
    return result

In [ ]:
all_combinations('00.1000000')

In [ ]:
# build W|0> = |who>
qr, qc = circ(6)
qc.h([0, 1, 3])
qc.cnot([0, 1, 3], [2, 5, 4])
W_gate = deepcopy(qc)

# build T|0> = |talks> = (|00>+|11>)/sqrt(2)
# build A|0> = |answers> = (|00>+|11>)/sqrt(2)
qr, qc = circ(2)
qc.h(0)
qc.cnot(0, 1)
T_gate = deepcopy(qc)
A_gate = deepcopy(qc)

# build U|0>=|psi>
qr, qc = circ(10)
qc.compose(W_gate, qr[:6], inplace=True)
qc.compose(T_gate, qr[6:8], inplace=True)
qc.compose(A_gate, qr[8:10], inplace=True)
# qc.barrier()

# contraction 1
qc.cnot(5, 6)
qc.h(5)
# qc.barrier()
# contraction 2
qc.cnot(4, 7)
qc.h(4)
# qc.barrier()
# contraction 3
qc.cnot(1, 8)
qc.h(1)
# qc.barrier()
# contraction 4
qc.cnot(0, 9)
qc.h(0)
# qc.barrier()

U_gate = deepcopy(qc)
Udag_gate = U_gate.reverse_ops()

print(isdagger(U_gate, Udag_gate))
# print(U_gate.draw())
# print(Udag_gate.draw())

In [ ]:
# build A
qr, qc = t.circ(2)
qc.h(0)
qc.cnot(0, 1)
A_circ = deepcopy(qc)
B_circ = deepcopy(qc)

qr, qc = t.circ(4)
qc.compose(A_circ, qr[:2], inplace=True)
qc.compose(B_circ, qr[-2:], inplace=True)
qc.cnot(0, 3)
qc.h(0)
U_circ = deepcopy(qc)
print(U_circ.draw())

In [ ]:
def amplification_analysis(U_gate, desired_str, M=100):
    # number of qubits, not counting with ancilla qubits
    n_qubits = U_gate.num_qubits
    
    # build Udag_gate
    Udag_gate = U_gate.reverse_ops()
    print(Udag_gate.draw())
    assert(isdagger(U_gate, Udag_gate))
    
    # build O_gate (oracle)
    assert(len(desired_str)==n_qubits)
    
    s_arr = np.array(list(desired_str), dtype=object)
    desired0 = list(np.where(s_arr == '0')[0])
    desired1 = list(np.where(s_arr == '1')[0])
    affected = list(np.where(s_arr != '.')[0])
    
    qr, O_gate = circ(n_qubits+1)
    O_gate.x(qr[desired0])
    O_gate.compose(MCMT('cx', len(affected), 1), qr[affected] + [qr[-1]], inplace=True)
    O_gate.x(qr[desired0])

    # build S0_gate (that flips sign of |0> state)
    qr, S0_gate = circ(n_qubits)
    S0_gate.x(qr)
    S0_gate.h(-1)
    S0_gate.compose(MCMT('cx', n_qubits-1, 1), qr, inplace=True)
    S0_gate.h(-1)
    S0_gate.x(qr)
    
    # build Q_gate (grover operator)
    qr, Q_gate = circ(n_qubits+1)
    Q_gate.compose(O_gate, qr, inplace=True)
#     Q_gate.barrier()
    Q_gate.compose(Udag_gate, qr[:-1], inplace=True)
#     Q_gate.barrier()
    Q_gate.compose(S0_gate, qr[:-1], inplace=True)
#     Q_gate.barrier()
    Q_gate.compose(U_gate, qr[:-1], inplace=True)
#     print(Q_gate.draw())

    # perform measure in U|0> = |psi> to measure a
    qr, qc = circ(n_qubits)
    qc.compose(U_gate, qr, inplace=True)
    qc.measure_all()

    shots = 2**16
    desired_dic = {}
    for bit, counts in t.counts(qc, shots=shots).items():
        if t.substr(bit, desired0) == '0'*len(desired0) and t.substr(bit, desired1) == '1'*len(desired1):
            desired_dic[bit] = desired_dic.get(bit, 0) + counts
            
    a = 0
    for bit, counts in desired_dic.items():
        a += counts/shots
    print(f'Estimated value of a = {a}')
    
    # build list of circuits
    circ_lst = []
    qr = qk.QuantumRegister(n_qubits+1, 'q')
    cr = qk.ClassicalRegister(n_qubits, 'c')
    qc = qk.QuantumCircuit(qr, cr)
    qc.compose(U_gate, qr[:-1], inplace=True)
    qc.x(-1)
    qc.h(-1)
    
    shots = 2**13
    for m in range(M):
        qc.compose(Q_gate, qr, inplace=True)
        qc_measured = deepcopy(qc)
        qc_measured.measure(qr[:-1], cr)
        circ_lst.append(qc_measured)
        
    all_counts = t.counts(circ_lst, shots=shots)
    desired_basis_states = all_combinations(desired_str)
    
    a_arr = np.zeros(len(all_counts)+1)
    a_arr[0] = a
    for i, dic in enumerate(all_counts):
        for state in desired_basis_states:
            a_arr[i+1] += dic.get(state, 0)/shots
    
    return a_arr
    
# a_arr = amplification_analysis(U_gate, '00.1000000', M=100)
# a_arr = amplification_analysis(U_gate, '00.1000000', M=20)
# a_arr
a_arr = amplification_analysis(U_circ, '0.10', M=100)
a_arr

In [ ]:
def amplification_analysis1(U_gate, desired_str, M=100):
    # number of qubits, not counting with ancilla qubits
    n_qubits = U_gate.num_qubits
    
    # build Udag_gate
    Udag_gate = U_gate.reverse_ops()
    assert(isdagger(U_gate, Udag_gate))
    
    # build O_gate (oracle)
    assert(len(desired_str)==n_qubits)
    
    s_arr = np.array(list(desired_str), dtype=object)
    desired0 = list(np.where(s_arr == '0')[0])
    desired1 = list(np.where(s_arr == '1')[0])
    affected = list(np.where(s_arr != '.')[0])
    
    O_gate = t.oracle(desired_str)
#     print(O_gate.draw())

    # build S0_gate (that flips sign of |0> state)
    qr, S0_gate = circ(n_qubits)
    S0_gate.x(qr)
    S0_gate.h(-1)
    S0_gate.compose(MCMT('cx', n_qubits-1, 1), qr, inplace=True)
    S0_gate.h(-1)
    S0_gate.x(qr)
#     print(S0_gate.draw())
    
    # build Q_gate (grover operator)
    qr, Q_gate = circ(n_qubits)
    Q_gate.compose(O_gate, qr, inplace=True)
#     Q_gate.barrier()
    Q_gate.compose(Udag_gate, qr, inplace=True)
#     Q_gate.barrier()
    Q_gate.compose(S0_gate, qr, inplace=True)
#     Q_gate.barrier()
    Q_gate.compose(U_gate, qr, inplace=True)
#     print(Q_gate.draw())

    # perform measure in U|0> = |psi> to measure a
    qr, qc = circ(n_qubits)
    qc.compose(U_gate, qr, inplace=True)
    qc.measure_all()

    shots = 2**16
    desired_dic = {}
    for bit, counts in t.counts(qc, shots=shots).items():
        if t.substr(bit, desired0) == '0'*len(desired0) and t.substr(bit, desired1) == '1'*len(desired1):
            desired_dic[bit] = desired_dic.get(bit, 0) + counts
            
    a = 0
    for bit, counts in desired_dic.items():
        a += counts/shots
    print(f'Estimated value of a = {a}')
    
    # build list of circuits
    circ_lst = []
    qr, cr, qc = t.circ_measured(n_qubits, n_qubits)
    qc.compose(U_gate, qr, inplace=True)
#     print(qc.draw())
    
    shots = 2**13
    for m in range(M):
        qc.compose(Q_gate, qr, inplace=True)
        qc_measured = deepcopy(qc)
        qc_measured.measure(qr, cr)
        circ_lst.append(qc_measured)
        
    all_counts = t.counts(circ_lst, shots=shots)
    desired_basis_states = all_combinations(desired_str)
    
    a_arr = np.zeros(len(all_counts)+1)
    a_arr[0] = a
    for i, dic in enumerate(all_counts):
        for state in desired_basis_states:
            a_arr[i+1] += dic.get(state, 0)/shots
    
    return a_arr
#     return 
    
# a_arr = amplification_analysis1(U_gate, '00.1000000', M=100)
# a_arr

In [ ]:
def plot(a_arr):
    a = a_arr[0]
    print(a)
    theta = np.arcsin(np.sqrt(a))
    m = np.arange(len(a_arr))
    ms = np.linspace(m[0], m[-1], 1000)
    
    fig = plt.figure(figsize = (7, 4))
    plt.xlabel(r'$m$')
    plt.ylabel('Prob of measuring good basis state')
    plt.plot(m, a_arr, 'o', label='experimental')
#     plt.plot(m, np.sin((2*m+1)*theta)**2, label='theoretical')
    plt.plot(ms, np.sin((2*ms+1)*theta)**2, label='theoretical')
    plt.axvline(x=np.floor(np.pi/(4*theta)), c='k', ls='--', label=r'$m_{floor}$')
    plt.axhline(y=max(a, 1-a), c='b', ls='--', label=r'max$(1-a,a)$')
    plt.legend()
    plt.savefig(get_filename_png(), format='png', dpi=600)
    plt.plot()
    return
plot(a_arr[:50])

In [ ]:
def write_arr_to_file(arr):
    with open(get_filename_txt(), 'w') as f:
        for i, a in enumerate(arr):
            f.write(f'{i} {a}\n')
write_arr_to_file(a_arr)